# File to generate Nadir OSSE tracks 

In [ ]:
import numpy as np 
import xarray as xr
import glob 
import os 
from scipy import interpolate
import random 
import datetime

import matplotlib.pyplot as plt

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy

import gc

from joblib import Parallel
from joblib import delayed as jb_delayed


## 1. - Functions 

In [ ]:
def open_satellite_data(date,str_satellite):

    dir_input = "/bettik/bellemva/satellite_track"

    date_split = date.astype('str').split("-")

    path_obs = os.path.join(dir_input,str_satellite,"2019","dt_global_*"+date_split[1]+date_split[2]+"*.nc")

    ds = xr.open_mfdataset(path_obs,combine='nested',concat_dim="time")

    empty_array = xr.DataArray(np.empty(ds.time.shape), dims=['time'])

    track = xr.Dataset({'point':(('time'),empty_array.data)},
                       coords={'time':('time',ds.time.values),'latitude':('latitude',ds.latitude.values),'longitude':('longitude',ds.longitude.values)})
    
    new_time_array = np.array([np.datetime64(t.astype('str').replace("2019","2012")).astype('datetime64[s]') for t in track.time.values])
    new_time_array_float = np.array([t.astype('float64') for t in new_time_array])
    track = track.assign_coords({"time":new_time_array_float})

    return track, new_time_array


In [ ]:
def open_model_data(date):

    dir_input_ssh = "/bettik/bellemva/MITgcm/MITgcm_filtered_final"

    path = os.path.join(dir_input_ssh,"MITgcm_filt_"+date.astype('str').replace("-","")+"*.nc")

    ds_ssh = xr.open_mfdataset(path,combine='nested',concat_dim="time")

    time_array_float = np.array([t.astype("datetime64[s]").astype('float64') for t in ds_ssh.time.values])
    ds_ssh = ds_ssh.assign_coords({"time":time_array_float})


    finterp = interpolate.RegularGridInterpolator([ds_ssh.time.values,ds_ssh.latitude.values,ds_ssh.longitude.values],
                                              ds_ssh.ssh_cor.values,
                                              bounds_error=False)
    
    return ds_ssh, finterp 

In [ ]:
def interp_satellite(track,new_time_array,interp):
    ssh_interp = interp(np.transpose([track.time.values, track.latitude.values,track.longitude.values]))

    ds = xr.Dataset({'ssh_cor':(('time'),ssh_interp[np.invert(np.isnan(ssh_interp))]),}, # don't forger to add 'time',
                    coords={'time':('time',new_time_array[np.invert(np.isnan(ssh_interp))]),
                            'latitude':('time',track.latitude.values[np.invert(np.isnan(ssh_interp))]),
                            'longitude':('time',track.longitude.values[np.invert(np.isnan(ssh_interp))])} 
                    )
    
    return ds 


In [ ]:
def create_dataset (date):

    ds_ssh, finterp = open_model_data(date)

    for sat in ["alg","c2","j3","s3a","s3b"]:
        
        track, new_time_array = open_satellite_data(date,sat)

        ds = interp_satellite(track,new_time_array,finterp)

        ds.to_netcdf("/bettik/bellemva/2022a_CCS_CalVal/nadir/"+sat+"/ds_"+sat+"_"+date.astype('str').replace("-","")+".nc")

        del track, ds, new_time_array 
        #gc.collect()
    
    del ds_ssh, finterp 
    #gc.collect()
    
    print(date,"done")

    return None 

## 2. - Track interpolating 

In [ ]:
dates = np.arange(np.datetime64("2012-06-02"),np.datetime64("2012-07-01"))
#dates = np.arange(np.datetime64("2012-06-04"),np.datetime64("2012-06-04"))

for d in dates : 
    create_dataset(d)

In [ ]:
longitude = np.arange(180,245.01,1/48)
latitude = np.arange(10,45.01,1/48)

fig, ax = plt.subplots(figsize=(9,4),dpi=200,subplot_kw={"projection": ccrs.PlateCarree(central_longitude=0.)})

ax.set_extent([-180, -115,10, 45],crs=ccrs.PlateCarree())
map_diff = ax.scatter(ds_alg.longitude.values,
                      ds_alg.latitude.values,
                      c=ds_alg.ssh_cor.values,cmap="Spectral",vmin=-1,vmax=1)

ax.add_feature(cfeature.LAND,color="lightgrey",zorder=1)
ax.add_feature(cfeature.OCEAN,color="black",zorder=0)
ax.coastlines(lw=0.5)
ax.set_aspect("equal")
ax.set_title("Altika OSSE tracks")

gridlines = ax.gridlines(
    alpha=0.3,
    color="white",
    draw_labels=True,
    xlocs=np.arange(-180,-115,10),
    ylocs=np.arange(10,45,5),
    # xformatter=cartopy.mpl.ticker.LongitudeFormatter(zero_direction_label=True,),
)
gridlines.right_labels = False
gridlines.top_labels  = False
gridlines.xlabel_style = {'size': 11}
gridlines.ylabel_style = {'size': 11}
ax.set_aspect("equal")
